### Downloading data from XP platform

Markos Flavio B. G. O.

__Context: Web scraping.__

In this notebook fund's context and historical information will be scrapped from data available at https://institucional.xpi.com.br/investimentos/fundos-de-investimento/lista-de-fundos-de-investimento.aspx and saved in a pickle file.
    
__Specific objectives__

     1. Download and save all useful information from funds at https://institucional.xpi.com.br/investimentos/fundos-de-investimento/lista-de-fundos-de-investimento.aspx in a pickle file.

__Status: Further work__
    - Include a function that updates and automatically includes new data into the existing data base.
    - Add parallelism.

In [99]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import pickle
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time
import json
import datetime
from sys import getsizeof
import pprint

The scraping strategy has two main parts:
    1. Gathering surface information from each fund at the main page of funds and saving them in a dictionary.
    2. Scanning each individual link and scraping deep information, including all historical data, and feeding them into the dictionaries.

#### 1. Gathering surface data

The surface data is enough for updating existing information. Thus, one may not scan the individual pages for updates. Some funds don't have historical data available. These are the ones with the 'Rent mês' attribute equals N/D or 'Não há'. That's the purpose of the 'Info' attribute.

In [11]:
main_page = 'https://institucional.xpi.com.br/investimentos/fundos-de-investimento/lista-de-fundos-de-investimento.aspx'

In [127]:
def surf_data(num_rows=''):
    """
    It scraps surface data from funds available at the link below and saves them as a pickle file.
    https://institucional.xpi.com.br/investimentos/fundos-de-investimento/lista-de-fundos-de-investimento.aspx

    Parameters
    ----------
    num_rows : int
        Number of funds to be scraped. If empty, all rows will be scanned.
    
    Returns
    -------
    df_surf : pandas.DataFrame
        Surface data.
    
    """ 
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("safebrowsing-disable-extension-blacklist")
    driver = webdriver.Chrome(options=options, executable_path=r'C:\chromedriver.exe')
    driver.get('https://institucional.xpi.com.br/investimentos/fundos-de-investimento/lista-de-fundos-de-investimento.aspx')

    if num_rows:
        rows = driver.find_elements_by_xpath("//table[@id='tableTodos']/tbody/tr")[:num_rows]
    else:
        rows = driver.find_elements_by_xpath("//table[@id='tableTodos']/tbody/tr")
   
    total_rows = len(rows)
    funds = []
    start_time = time.time()
    for i, row in enumerate(rows):
        d = {}
        d['Risco'] = row.find_element_by_xpath(".//div").get_attribute('class').split(' ')[1]
        d['Classifiacao MorningStar'] = row.find_element_by_xpath(".//td[starts-with(@id, 'rptFundos_tdMorningstar')]").get_attribute("data-order")
        infos = row.find_elements_by_xpath(".//td")
        d['Nome'] = row.find_element_by_xpath(".//td[starts-with(@id, 'rptFundos_tdNomeFundo')]").text
        print('{0}/{1}: {2}'.format(i+1, total_rows, d['Nome']))
        
        d['Aplicacao minima'] = row.find_element_by_xpath(".//td[starts-with(@id, 'rptFundos_tdAplicacaoMin')]").get_attribute("data-order").split(',')[0]
        d['Taxa adm. a.a. (%)'] = infos[4].text[:-1].replace(',', '.')
        d['Resgate (D+)'] = infos[5].text[-1].replace(',', '.')
        
        if (infos[6].text == 'N/D') or (infos[6].text == 'Não há'): # it's enough to check infos[6]
            d['Rent. mes'] = np.nan
            d['Rent. ano'] = np.nan
            d['Rent. 12M'] = np.nan
            d['Info'] = 0
        else:
            d['Rent. mes'] = infos[6].text[:-1].replace(',', '.')
            d['Rent. ano'] = infos[7].text[:-1].replace(',', '.')
            d['Rent. 12M'] = infos[8].text[:-1].replace(',', '.')
            d['Info'] = 1
        
        status = infos[10].find_element_by_xpath(".//div").get_attribute('class')
        if 'icon-lock' in status:
            d['Status'] = 'Fechado'
        else:  #'btn-group pull-right
            d['Status'] = 'Aberto'
        d['Codigo'] = infos[9].find_element_by_xpath(".//button").get_attribute("data-codigo")
        #d['Pagina'] = 'https://institucional.xpi.com.br/investimentos/fundos-de-investimento/detalhes-de-fundos-de-investimento.aspx?F='+d['Codigo']
        
        # getting the details
        d['Data de inicio'] = row.get_attribute('data-inicio')
        d['Patrimonio'] = row.get_attribute('data-patrimonio')[3:].replace('.', '').replace(',', '.')
        d['Patrimonio 12M'] = row.get_attribute('data-patrimonio12')[3:].replace('.', '').replace(',', '.')
        d['Data da cota'] = row.get_attribute('data-cota')
        d['Valor da cota'] = row.get_attribute('data-valor').replace(',', '.')
        d['Taxa perf. (%)'] = row.get_attribute('data-taxa-performance').replace(',', '.')
        d['Benchmark'] = row.get_attribute('data-benchmark').strip()
        
        funds.append(d)
        
        # estimating remaining time
        diff = time.time() - start_time
        avg_time = diff/(i+1)
        estimated_remaning_time = avg_time*(total_rows-(i+1))
        print('Estimated remaining time (s):', estimated_remaning_time)
    
    driver.quit() 
    
    now = datetime.datetime.now()
    date = now.strftime("%d-%m-%Y")
    file_name = 'xp_funds_surf'
    with open('{0}_{1}.pickle'.format(file_name, date), 'wb') as handle:
        pickle.dump(pd.DataFrame(funds), handle, protocol=pickle.HIGHEST_PROTOCOL)  
    
    return pd.DataFrame(funds)

In [128]:
df_surf = surf_data()

1/409: AZ Quest Termo FIRF
Estimated remaining time (s): 77.19512557983398
2/409: Bradesco Asset FIC FIRF CP LP Plus
Estimated remaining time (s): 77.28116548061371
3/409: CA Indosuez DI Master FI Renda Fixa Referenciado DI LP
Estimated remaining time (s): 73.70647112528482
4/409: Mongeral Aegon Renda Fixa Fundo de Investimento
Estimated remaining time (s): 73.31252932548523
5/409: SulAmérica Exclusive FI Referenciado DI
Estimated remaining time (s): 71.53373622894287
6/409: Trend DI Simples FIRF
Estimated remaining time (s): 72.57956492900848
7/409: Trend Pós-Fixado FIRF Simples
Estimated remaining time (s): 71.50908034188407
8/409: Absolute Alpha FIC FIM
Estimated remaining time (s): 70.70762497186661
9/409: AF Invest Geraes FIRF CP
Estimated remaining time (s): 69.87482706705728
10/409: ARX Denali FIC FIRF CP
Estimated remaining time (s): 69.36644961833954
11/409: ARX Vinson FIRF CP
Estimated remaining time (s): 68.9660096168518
12/409: Az Quest Low Vol FIM
Estimated remaining time 

Estimated remaining time (s): 62.04154814067094
93/409: Kinea Chronos FIM
Estimated remaining time (s): 61.97033708839007
94/409: Mauá Institucional FIC FIM
Estimated remaining time (s): 61.88833263326198
95/409: Navi Long Short FIC FIM
Estimated remaining time (s): 61.80946299402338
96/409: Novus Institucional FIC FIM
Estimated remaining time (s): 61.721508257091045
97/409: Occam Equity Hedge FIC FIM
Estimated remaining time (s): 61.64768724343211
98/409: Occam Institucional FIC FIM II
Estimated remaining time (s): 61.573278867468545
99/409: Opportunity Market FIC DE FIM
Estimated remaining time (s): 61.48545204991042
100/409: PIMCO Income FIC FIM IE
Estimated remaining time (s): 61.40474662542343
101/409: Riza Daikon Advisory FIC FIM CP
Estimated remaining time (s): 61.19938720098817
102/409: Schroder High Grade Advisory FIRF CP
Estimated remaining time (s): 61.1192362168256
103/409: Schroder Liquid Alternatives IE FIM
Estimated remaining time (s): 61.03402533114536
104/409: Selectio

Estimated remaining time (s): 52.46294132780634
182/409: MZK Dinâmico Advisory FIC FIM
Estimated remaining time (s): 52.321716354443474
183/409: Neo Multi Estratégia 30 FIC FIM
Estimated remaining time (s): 52.18726446328919
184/409: Neuberger Berman US Multi Cap Opportunities Advisory FIM IE
Estimated remaining time (s): 51.99472816741985
185/409: Novus Macro FIC FIM
Estimated remaining time (s): 51.85446868071685
186/409: Oaktree Global Credit FIC FIM IE
Estimated remaining time (s): 51.71950234264456
187/409: Occam Retorno Absoluto Advisory FIC FIM
Estimated remaining time (s): 51.57777280603501
188/409: Occam Retorno Absoluto FIC FIM
Estimated remaining time (s): 51.43140524245323
189/409: Opportunity Total FIC de FIM
Estimated remaining time (s): 51.31378246993615
190/409: Pacifico Macro FIC FIM
Estimated remaining time (s): 51.1516796237544
191/409: Perfin Equity Hedge FIC FIM
Estimated remaining time (s): 50.990016924773215
192/409: Persevera Compass Advisory FIC FIM
Estimated r

Estimated remaining time (s): 35.395702704405174
274/409: Geo Empresas Globais FIC FIA IE
Estimated remaining time (s): 35.15606690497294
275/409: Giant Zarathustra FIC FIM
Estimated remaining time (s): 34.914421738711276
276/409: Hashdex Criptoativos Discovery FIC FIM
Estimated remaining time (s): 34.6740497307501
277/409: HIX Capital Advisory FIC FIA
Estimated remaining time (s): 34.43432084920174
278/409: HIX Capital FIA
Estimated remaining time (s): 34.19222632329241
279/409: Ibiuna Equities 30 FIC FIA
Estimated remaining time (s): 33.9510946461804
280/409: Ibiuna Long Biased FIC FIM
Estimated remaining time (s): 33.71167797446251
281/409: Ibiuna Long Short Advisory FIC FIM
Estimated remaining time (s): 33.46968844267821
282/409: Ibiuna Long Short STLS FIC FIM
Estimated remaining time (s): 33.23449850251489
283/409: Icatu Vanguarda Dividendos 30 FIA
Estimated remaining time (s): 32.99280594262975
284/409: Icatu Vanguarda Dividendos FIA
Estimated remaining time (s): 32.7500716691285

Estimated remaining time (s): 11.122838100443689
370/409: Trend XP Inc Balanceado FIC FIM
Estimated remaining time (s): 10.85434366561271
371/409: Trend XP Inc FIC FIA IE
Estimated remaining time (s): 10.5862074181076
372/409: Truxt I Valor Institucional FIC FIA
Estimated remaining time (s): 10.316604289957272
373/409: Truxt Long Bias Advisory FIC FIM
Estimated remaining time (s): 10.047290781547813
374/409: Valora Guardian Advisory FIC FIM CP
Estimated remaining time (s): 9.777316186517318
375/409: Velt 120 Advisory FIC FIA
Estimated remaining time (s): 9.501168294270833
376/409: Verde AM Long Bias FIC FIA
Estimated remaining time (s): 9.230722736805044
377/409: Vinci Crédito Estruturado Selection Advisory FIC FIM Crédito Privado
Estimated remaining time (s): 8.959797846543694
378/409: Vinci Mosaico FIA
Estimated remaining time (s): 8.687559123392457
379/409: Vinci Seleção FIA
Estimated remaining time (s): 8.415068447747142
380/409: Vinland Long Bias Advisory FIC FIM
Estimated remaini

In [129]:
df_surf[100:105]

,Risco,Classifiacao MorningStar,Nome,Aplicacao minima,Taxa adm. a.a. (%),Resgate (D+),Rent. mes,Rent. ano,Rent. 12M,Info,Status,Codigo,Data de inicio,Patrimonio,Patrimonio 12M,Data da cota,Valor da cota,Taxa perf. (%),Benchmark
100,RiscoMedio,0,Riza Daikon Advisory FIC FIM CP,5000,0.80,0,NaN,NaN,NaN,0,Aberto,800952,30/03/2020,14508766.52,7576182.05,14/08/2020,1.07666170,20.00,CDI
101,RiscoMedio,0,Schroder High Grade Advisory FIRF CP,1000,0.60,0,0.34,-5.37,-3.95,1,Aberto,800610,26/12/2018,137446650.72,267747826.58,14/08/2020,1.00998387,-,CDI
102,RiscoMedio,0,Schroder Liquid Alternatives IE FIM,5000,1.00,7,0.67,-0.55,2.72,1,Aberto,2862,22/09/2016,15097149.06,19001228.52,14/08/2020,1.22930188,-,CDI
103,RiscoMedio,0,Selection Debentures Incentivadas Hedge FIC FI...,100,0.50,0,0.26,0.52,-0.32,1,Aberto,800702,26/04/2019,101480316.39,193733082.09,13/08/2020,1.01090745,20.00,CDI
104,RiscoMedio,2,Selection Multimercado FIC FIM,100,0.75,0,0.17,-1.49,1.20,1,Aberto,2813,25/08/2016,125695269.59,99724927.81,13/08/2020,1.32009703,20.00,CDI


In [124]:
getsizeof(df_surf)/1000000 # size in MBs

0.483767

#### 2. Getting detailed information

First, we'll walk through the page, testing whether the XPath commands are working in particular fund. Then, we'll build a function combining all commmands that will be used for scraping.

In [80]:
fund_code = 2862
fund_page = 'https://institucional.xpi.com.br/investimentos/fundos-de-investimento/detalhes-de-fundos-de-investimento.aspx?F='+str(fund_code)
options = webdriver.ChromeOptions()
#options.add_argument("--headless")
driver = webdriver.Chrome(options=options, executable_path=r'C:\chromedriver.exe')
driver.get(fund_page)

All information below are not necessary as them are available at the main page.

In [82]:
infos = driver.find_elements_by_xpath("//table[@class='table table-bordered table-hover tabelaXP']/tbody/tr/td")
d = {}
d['Data'] = infos[0].text
d['Cota'] = infos[1].text.replace(',', '.')
d['Rent. dia'] = infos[2].text[:-1].replace(',', '.')
d['PL atual (R$)'] = infos[6].text[2:].replace('.', '').replace(',', '.')
d['PL medio anual (R$)'] = infos[7].text[2:].replace('.', '').replace(',', '.')
print(d)

{'Data': '14/08/2020', 'Cota': '1.22930188', 'Rent. dia': '-0.04', 'PL atual (R$)': ' 15097149.06', 'PL medio anual (R$)': ' 19001228.52'}


In [98]:
d = {}
driver.find_element_by_xpath("//a[text()='Características do Fundo']").click()
time.sleep(1) # add a wait here
# d['Inicio'] = driver.find_element_by_xpath("//div[@class='tab-pane fade active in']/table/tbody/tr[1]/td[2]/h4").text
d['Objetivos'] = driver.find_element_by_xpath("//strong[text()='Objetivos']/ancestor::p/following-sibling::p").text.strip()
d['Publico-alvo'] = driver.find_element_by_xpath("//strong[text()='Público-alvo']/ancestor::p/following-sibling::p").text.strip()
d['Politica de gestao'] = driver.find_element_by_xpath("//strong[text()='Política de gestão']/ancestor::p/following-sibling::p").text.strip()

# parsing file links
files = driver.find_elements_by_xpath("//div[@id='dvArqsDownload']/div")
links = {}
for file in files:
    link = file.find_element_by_xpath(".//p/a").get_attribute('href')
    name = file.find_element_by_xpath(".//p/a").get_attribute('id')
    links[name] = link
d['Links'] = links
print(d)

{'Objetivos': 'O objetivo do FUNDO é gerar retornos acima do CDI e baixa correlação com as principais classes de ativos domésticas. O produto oferece acesso à plataforma global de alternativos líquidos da Schroders, sendo seu retorno total é rentabilidade dos fundos investidos, combinada ao diferencial de taxa de juros (entre BRL e USD), menos o custo do hedge.', 'Publico-alvo': 'Investidores qualificados', 'Politica de gestao': 'O FUNDO buscará o crescimento de capital investindo, no mínimo, 67% (sessenta e sete por cento) de seu patrimônio líquido em fundos domiciliado no exterior', 'Links': {'lnkProspecto': 'https://institucional.xpi.com.br/downloads/xpgestao/Prospecto/Prospecto_2862_v1.pdf', 'lnkRegulamento': 'https://institucional.xpi.com.br/downloads/xpgestao/Regulamento/Regulamento_24018821000113_v3.pdf', 'lnkLamina': 'https://institucional.xpi.com.br/downloads/xpgestao/Lamina/Lamina_24018821000113_v4.pdf', 'lnkCaracteristica': 'https://institucional.xpi.com.br/downloads/xpgesta

In [100]:
driver.find_element_by_xpath("//a[text()='Informações complementares']").click()
time.sleep(1) # add a wait here
d['Aplicação Inicial Mínima'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Aplicação Inicial Mínima')]/following-sibling::td").text[2:].strip().replace('.', '').replace(',', '.')
d['Movimentação Mínima'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Movimentação Mínima')]/following-sibling::td").text[2:].strip().replace('.', '').replace(',', '.')
d['Saldo de Permanência'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Saldo de Permanência')]/following-sibling::td").text[2:].strip().replace('.', '').replace(',', '.')
#d['Taxa de Administração'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Taxa de Administração')]/following-sibling::td").text[:-1].replace(',', '.')
d['Taxa Máxima de Administração'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Taxa Máxima de Administração')]/following-sibling::td").text[:-4].replace(',', '.')
d['Taxa de Performance'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Taxa de Performance')]/following-sibling::td").text.strip()
d['Tributação'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Tributação')]/following-sibling::td").text.strip()
d['IOF'] = driver.find_element_by_xpath("//td[starts-with(text(), 'IOF')]/following-sibling::td").text.strip()
d['Aplicação'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Aplicação\n')]/following-sibling::td").text.strip()
d['Resgate - Cotização'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Resgate - Cotização')]/following-sibling::td").text.strip()
d['Resgate - Liquidação Financeira'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Resgate - Liquidação Financeira')]/following-sibling::td").text.strip()
d['Horário para Aplicação e Resgate'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Horário para Aplicação e Resgate')]/following-sibling::td").text.strip()
d['Divulgação de quota e PL'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Divulgação de quota e PL')]/following-sibling::td").text.strip()
d['Classificação Anbima'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Classificação Anbima')]/following-sibling::td").text.strip()
d['Código Anbima'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Código Anbima')]/following-sibling::td").text.strip()
d['Custódia'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Custódia')]/following-sibling::td").text.strip()
d['Auditoria'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Auditoria')]/following-sibling::td").text.strip()
d['Gestor'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Gestor')]/following-sibling::td").text.strip()
d['Administrador'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Administrador\n')]/following-sibling::td").text.strip()

In [101]:
pprint.pprint(d)

{'Administrador': 'BNY Mellon Serviços Financeiros DTVM S.A.',
 'Aplicação': 'D+0 (Dias Úteis)',
 'Aplicação Inicial Mínima': '5000.00',
 'Auditoria': 'KPMG Auditores Independentes',
 'Classificação Anbima': '',
 'Custódia': 'BNY Mellon Banco S.A',
 'Código Anbima': '423998',
 'Divulgação de quota e PL': '',
 'Gestor': 'Schroder Investment Management',
 'Horário para Aplicação e Resgate': '14:30:00',
 'IOF': 'Começa com uma alíquota de 96% do rendimento e vai a ZERO para quem '
        'resgatar a partir do 30º dia da data da aplicação.',
 'Links': {'lnkCaracteristica': 'https://institucional.xpi.com.br/downloads/xpgestao/Caracteristica/Caracteristica_24018821000113_v79.pdf',
           'lnkLamina': 'https://institucional.xpi.com.br/downloads/xpgestao/Lamina/Lamina_24018821000113_v4.pdf',
           'lnkProspecto': 'https://institucional.xpi.com.br/downloads/xpgestao/Prospecto/Prospecto_2862_v1.pdf',
           'lnkRegulamento': 'https://institucional.xpi.com.br/downloads/xpgestao/Regu

Getting the historical performance data. This page was found by looking at the 'Network' tab inside the inspection tool of Chrome browser; specifically, inside the headers of a request url that was loaded inside the inner XHR tab.

In [103]:
hist_data = "https://institucional.xpi.com.br/api/Ranking/GraficoFundo?idFundo={0}&tipo=3".format(fund_code)
driver.get(hist_data) # 'tipo=3' holds the most lengthy data (at most 36 months)
soup = BeautifulSoup(driver.page_source)
dict_from_json = json.loads(soup.find("body").text)
graph = dict_from_json['Fundo']

# pasing data
for day in graph:
    del day['key']
    day['Date'] = day['Ano'].split('T')[0]
    del day['Ano']
print(graph[0])

In [118]:
d['Daily profitability'] = pd.DataFrame(graph).set_index('Date')

In [120]:
pprint.pprint(d)

{'Administrador': 'BNY Mellon Serviços Financeiros DTVM S.A.',
 'Aplicação': 'D+0 (Dias Úteis)',
 'Aplicação Inicial Mínima': '5000.00',
 'Auditoria': 'KPMG Auditores Independentes',
 'Classificação Anbima': '',
 'Custódia': 'BNY Mellon Banco S.A',
 'Código Anbima': '423998',
 'Daily profitability':                 Valor
Date                 
2017-08-18   0.000000
2017-08-21  -0.040417
2017-08-22   0.074214
2017-08-23   0.417737
2017-08-24   0.169065
...               ...
2020-08-07  10.398753
2020-08-10  10.216926
2020-08-11  10.157575
2020-08-12  10.472335
2020-08-13  10.467243

[744 rows x 1 columns],
 'Divulgação de quota e PL': '',
 'Gestor': 'Schroder Investment Management',
 'Horário para Aplicação e Resgate': '14:30:00',
 'IOF': 'Começa com uma alíquota de 96% do rendimento e vai a ZERO para quem '
        'resgatar a partir do 30º dia da data da aplicação.',
 'Links': {'lnkCaracteristica': 'https://institucional.xpi.com.br/downloads/xpgestao/Caracteristica/Caracteristica_24018

#### Putting all together into a single function

In [152]:
def deep_data(codes, infos01, saving=True):
    """
    It scraps detailed data from funds available at the link below and saves them as a pickle file.
    https://institucional.xpi.com.br/investimentos/fundos-de-investimento/lista-de-fundos-de-investimento.aspx
    It opens all individual links from the and scrap historical data and detailed information from funds.

    Parameters
    ----------
    codes : pandas.Series
        Holds the funds referece codes on the XP platform.
    infos : pandas.Series
        Holds a binary information for each fund that indicates if it has historical data (1) or not (0).
    
    Returns
    -------
    df_deep : pandas.DataFrame
        All detailed data from funds, including historical informaiton if available.
    
    """ 
    options = webdriver.ChromeOptions()
    options.add_argument("safebrowsing-disable-extension-blacklist")
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options, executable_path=r'C:\chromedriver.exe')
    
    total_rows = len(list(codes))
    start_time = time.time()
    funds = []
    for i, (code, info01) in enumerate(zip(codes, infos01)):
        d = {}
        #if i % 10 == 0:
        print(code)
        #print('Progress(%)', (i/len(list_funds))*100)
        
        
        driver.get('https://institucional.xpi.com.br/investimentos/fundos-de-investimento/detalhes-de-fundos-de-investimento.aspx?F='+str(code))
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.XPATH, "//table[@class='table table-bordered table-hover tabelaXP']/tbody/tr/td"))
            )
        finally:
            #infos = driver.find_elements_by_xpath("//table[@class='table table-bordered table-hover tabelaXP']/tbody/tr/td")
            #fund_dict['Data'] = infos[0].text
            #fund_dict['Cota'] = infos[1].text.replace(',', '.')
            
            #if fund_dict['Info'] == 1:
            #    fund_dict['Rent. dia'] = infos[2].text[:-1].replace(',', '.')
            #    fund_dict['PL atual (R$)'] = infos[6].text[2:].replace('.', '').replace(',', '.')
            #    fund_dict['PL medio anual (R$)'] = infos[7].text[2:].replace('.', '').replace(',', '.')
            #else:
            #    fund_dict['Rent. dia'] = np.nan
            #    fund_dict['PL atual (R$)'] = np.nan
            #    fund_dict['PL medio anual (R$)'] = np.nan
            
            driver.find_element_by_xpath("//a[text()='Características do Fundo']").click()
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.XPATH, "//div[@class='tab-pane fade active in']/table/tbody/tr[1]/td[2]/h4"))
                )
            finally:
                d['Inicio'] = driver.find_element_by_xpath("//div[@class='tab-pane fade active in']/table/tbody/tr[1]/td[2]/h4").text
                #if fund_dict['Info'] == 0: # another source of getting PL info if N/D data
                d['PL atual (R$)'] = driver.find_element_by_xpath("//div[@class='tab-pane fade active in']/table/tbody/tr[2]/td[2]/h4/span").text.replace('.', '').replace(',', '.')
                d['PL medio anual (R$)'] = driver.find_element_by_xpath("//div[@class='tab-pane fade active in']/table/tbody/tr[3]/td[2]/h4/span").text.replace('.', '').replace(',', '.')
                d['Objetivos'] = driver.find_element_by_xpath("//strong[text()='Objetivos']/ancestor::p/following-sibling::p").text.strip()
                d['Publico-alvo'] = driver.find_element_by_xpath("//strong[text()='Público-alvo']/ancestor::p/following-sibling::p").text.strip()
                d['Politica de gestao'] = driver.find_element_by_xpath("//strong[text()='Política de gestão']/ancestor::p/following-sibling::p").text.strip()
                
                files = driver.find_elements_by_xpath("//div[@id='dvArqsDownload']/div")
                links = {}
                for file in files:
                    link = file.find_element_by_xpath(".//p/a").get_attribute('href')
                    name = file.find_element_by_xpath(".//p/a").get_attribute('id')
                    links[name] = link
                d['Links'] = links
                
                driver.find_element_by_xpath("//a[text()='Informações complementares']").click()
                try:
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located(
                            (By.XPATH, "//td[starts-with(text(), 'Aplicação Inicial Mínima')]"))
                    )
                finally:
                    d['Aplicação Inicial Mínima'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Aplicação Inicial Mínima')]/following-sibling::td").text[2:].strip().replace('.', '').replace(',', '.')
                    d['Movimentação Mínima'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Movimentação Mínima')]/following-sibling::td").text[2:].strip().replace('.', '').replace(',', '.')
                    d['Saldo de Permanência'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Saldo de Permanência')]/following-sibling::td").text[2:].strip().replace('.', '').replace(',', '.')
                    d['Taxa de Administração'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Taxa de Administração')]/following-sibling::td").text[:-1].replace(',', '.')
                    d['Taxa Máxima de Administração'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Taxa Máxima de Administração')]/following-sibling::td").text[:-4].replace(',', '.')
                    d['Taxa de Performance'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Taxa de Performance')]/following-sibling::td").text.strip()
                    d['Tributação'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Tributação')]/following-sibling::td").text.strip()
                    d['IOF'] = driver.find_element_by_xpath("//td[starts-with(text(), 'IOF')]/following-sibling::td").text.strip()
                    d['Aplicação'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Aplicação\n')]/following-sibling::td").text.strip()
                    d['Resgate - Cotização'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Resgate - Cotização')]/following-sibling::td").text.strip()
                    d['Resgate - Liquidação Financeira'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Resgate - Liquidação Financeira')]/following-sibling::td").text.strip()
                    d['Horário para Aplicação e Resgate'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Horário para Aplicação e Resgate')]/following-sibling::td").text.strip()
                    d['Divulgação de quota e PL'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Divulgação de quota e PL')]/following-sibling::td").text.strip()
                    d['Classificação Anbima'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Classificação Anbima')]/following-sibling::td").text.strip()
                    d['Código Anbima'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Código Anbima')]/following-sibling::td").text.strip()
                    d['Custódia'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Custódia')]/following-sibling::td").text.strip()
                    d['Auditoria'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Auditoria')]/following-sibling::td").text.strip()
                    d['Gestor'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Gestor')]/following-sibling::td").text.strip()
                    d['Administrador'] = driver.find_element_by_xpath("//td[starts-with(text(), 'Administrador\n')]/following-sibling::td").text.strip()
                    
                    if info01 == 1:
                        driver.get("https://institucional.xpi.com.br/api/Ranking/GraficoFundo?idFundo={0}&tipo=3".format(code))
                        time.sleep(0.1)
                        soup = BeautifulSoup(driver.page_source)
                        dict_from_json = json.loads(soup.find("body").text)
                        graph = dict_from_json['Fundo']
                        for day in graph:
                            del day['key']
                            day['Date'] = day['Ano'].split('T')[0]
                            del day['Ano']
                        d['Daily profitability'] = pd.DataFrame(graph).set_index('Date')
                    else:
                        d['Daily profitability'] = np.nan
        funds.append(d)
        
        # estimating remaining time
        diff = time.time() - start_time
        avg_time = diff/(i+1)
        estimated_remaning_time = avg_time*(total_rows-(i+1))
        print('Estimated remaining time (min):', estimated_remaning_time/60)
        
    driver.quit() 
    
    now = datetime.datetime.now()
    date = now.strftime("%d-%m-%Y")
    file_name = 'xp_funds_deep'
    with open('{0}_{1}.pickle'.format(file_name, date), 'wb') as handle:
        pickle.dump(pd.DataFrame(funds), handle, protocol=pickle.HIGHEST_PROTOCOL)  
            
    return pd.DataFrame(funds)        

In [153]:
df_deep = deep_data(df_surf['Codigo'], df_surf['Info'])

800981
Estimated remaining time (min): 28.129416751861573
800868
Estimated remaining time (min): 23.503977715969086
2289
Estimated remaining time (min): 22.320418294270834
2900
Estimated remaining time (min): 21.664541512727737
800441
Estimated remaining time (min): 20.87072399457296
800832
Estimated remaining time (min): 20.189858568376962
2798
Estimated remaining time (min): 20.083855261121478
7455
Estimated remaining time (min): 19.866573802630107
2692
Estimated remaining time (min): 19.637232709814
800590
Estimated remaining time (min): 19.448500771522518
800588
Estimated remaining time (min): 19.255462106791413
3058
Estimated remaining time (min): 19.11986757583088
800817
Estimated remaining time (min): 18.966872185927173
2732
Estimated remaining time (min): 18.907213906447094
2676
Estimated remaining time (min): 18.802672997580636
2149
Estimated remaining time (min): 18.771623787283897
800456
Estimated remaining time (min): 18.797063993940167
800690
Estimated remaining time (min)

Estimated remaining time (min): 12.747927440519202
800754
Estimated remaining time (min): 12.694656089428062
800758
Estimated remaining time (min): 12.648533712367755
800935
Estimated remaining time (min): 12.595396746458357
800457
Estimated remaining time (min): 12.540329797506331
6006
Estimated remaining time (min): 12.492647225019947
6111
Estimated remaining time (min): 12.44177602103405
800801
Estimated remaining time (min): 12.390408105787888
800574
Estimated remaining time (min): 12.345887341669627
800959
Estimated remaining time (min): 12.284398798429837
800691
Estimated remaining time (min): 12.231707531901508
1017
Estimated remaining time (min): 12.183392095869515
50110
Estimated remaining time (min): 12.13560723896268
27744
Estimated remaining time (min): 12.083773924119816
800757
Estimated remaining time (min): 12.033940739557147
800634
Estimated remaining time (min): 11.982488010240639
800767
Estimated remaining time (min): 11.930191571158145
2886
Estimated remaining time (

Estimated remaining time (min): 5.700159102637474
2738
Estimated remaining time (min): 5.651129119632187
800557
Estimated remaining time (min): 5.600590225452739
800860
Estimated remaining time (min): 5.551108429795605
2974
Estimated remaining time (min): 5.502122810174217
800466
Estimated remaining time (min): 5.452606476677789
800495
Estimated remaining time (min): 5.403983961935811
2883
Estimated remaining time (min): 5.3577434073058
800509
Estimated remaining time (min): 5.308274886714088
800682
Estimated remaining time (min): 5.258343149736474
800946
Estimated remaining time (min): 5.207246310347753
2890
Estimated remaining time (min): 5.157591977869585
800674
Estimated remaining time (min): 5.106982602296691
800878
Estimated remaining time (min): 5.056058481664606
800911
Estimated remaining time (min): 5.005645714203516
2635
Estimated remaining time (min): 4.955650556592289
2718
Estimated remaining time (min): 4.905824512398088
800692
Estimated remaining time (min): 4.85638367962

In [154]:
getsizeof(df_deep)/1000000 # size in MBs

18.181227